In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys 
sys.path.insert(0, '/content/drive/MyDrive/CBN_Automate')
%cd /content/drive/MyDrive/CBN_Automate

import pandas as pd
import Resource  as res
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import gspread_dataframe as gd
from googleapiclient import discovery
import psycopg2
import psycopg2.extras as extras

connection = psycopg2.connect(user="postgres",
                              password="smartsetter",
                              host="ss-db-data-dev.cpeist8s9qou.us-west-2.rds.amazonaws.com",
                              port="5432",
                              database="postgres")

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CBN_Automate/cbn-file-Automate.json",scope)
service = discovery.build('sheets', 'v4', credentials=credentials)

/content/drive/MyDrive/CBN_Automate


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
#finding mobile numbers using name
def finding_mobile_name(name,state, ph):
    
    
    if ph == None:
        
        if str(name).lower() != 'nan' and str(state).lower() != 'nan':
            name_file = globals()[f"{state}"][globals()[f"{state}"]["name"]==name]
            if len(name_file) > 0:
                name_file = name_file.reset_index(drop = True)
                for name_index , manual_name in enumerate(name_file["name"]):
                    if str(name).lower() == str(manual_name).lower():
                        if str(name_file.loc[name_index,'phone'])[0] == '1' and len(str(name_file.loc[name_index,'phone'])) < 11:
                            return None
                        else:
                            return name_file.loc[name_index,'phone']

    else:
        return ph
    
#finding mobile numbers using email
def finding_mobile_email(email, state, ph):
    
    if ph == None:
        if str(email).lower() != 'nan'and str(state).lower() != 'nan':
            email_file = globals()[f"{state}"][globals()[f"{state}"]["email"]==email]
            if len(email_file) > 0:
                email_file = email_file.reset_index(drop = True)
                
                for email_index , manual_email in enumerate(email_file["email"]):
                    if str(email).lower() == str(manual_email).lower():
                        if str(email_file.loc[email_index,'phone'])[0] == '1' and len(str(email_file.loc[email_index,'phone'])) < 11:
                            return None
                        else:
                            return email_file.loc[email_index,'phone']
            
    else:
        return ph
    
# #finding mobile numbers using name
# def finding_mobile_name(name, state, ph , Zip):
    

#     if ph == None and str(Zip).lower()!='nan' and Zip != None and str(state).lower() != 'nan':
        
#         Zip = str(Zip).replace(".0","")
#         Zip_file =  globals()[f"{state}"][globals()[f"{state}"]["zip"] == str(Zip)]    
#         Zip_file.reset_index(drop = True , inplace = True)
        
#         if str(name).lower() != 'nan' and str(state).lower() != 'nan':
#             for name_index , manual_name in enumerate(Zip_file["name"]):
#                 if str(name).lower() == str(manual_name).lower():
#                     return Zip_file.loc[name_index,'phone']
            
#     else:
#         return ph
    
# #finding mobile numbers using name
# def finding_mobile_email(email, state, ph , Zip):
    
#     if ph == None and str(Zip).lower() != 'nan' and Zip != None and str(state).lower() != 'nan':
        
#         Zip = str(Zip).replace(".0","")
#         Zip_file =  globals()[f"{state}"][globals()[f"{state}"]["zip"]== str(Zip)]    
#         Zip_file.reset_index(drop = True , inplace = True)
        
#         if str(email).lower() != 'nan'and str(state).lower() != 'nan':
#             for email_index , manual_email in enumerate(Zip_file["email"]):
#                 if str(email).lower() == str(manual_email).lower():
#                     return Zip_file.loc[email_index,'phone']
            
#     else:
#         return ph

###  Input URL'S

In [ ]:
### Input file Url 
urls = ["https://docs.google.com/spreadsheets/d/1xPv4ihEPT8ViJeeNFfPnxrQFc5QAqRk79OS4aCiC0Dk/edit?urlBuilderDomain=smartsetter.io#gid=1367061075",
        "https://docs.google.com/spreadsheets/d/1VGxx3f8-wf0X2RaRh7-LQDiiEWBt5caPlp0ytjNPD4E/edit#gid=1844540345",
        "https://docs.google.com/spreadsheets/d/1Bv8fU8O-m6exp7QoqUn1fGUp3dd-lCKFYbQD7KY9Lq0/edit?urlBuilderDomain=smartsetter.io#gid=307053804",
        "https://docs.google.com/spreadsheets/d/1AvUaLkzRxksWb4IuFjWD_3NTlqMB--0ow0YYQ0DTrJM/edit?urlBuilderDomain=smartsetter.io#gid=228921234",
        "https://docs.google.com/spreadsheets/d/1R4T7G5-I6hkOcNBXovokeJLewNSpgU1mpMAb0x9GOlk/edit?urlBuilderDomain=smartsetter.io#gid=246111282",
        "https://docs.google.com/spreadsheets/d/1UhAUW_3jr3aWMZtlybIKH_MYLYWKe2xChMXPlj-hcWI/edit?urlBuilderDomain=smartsetter.io#gid=1696486497",
        "https://docs.google.com/spreadsheets/d/1GXy2pdYhbuyku2wQjrVMbfwJr7nccdt5_mZ1V_PteRM/edit?urlBuilderDomain=smartsetter.io#gid=382055332",
        "https://docs.google.com/spreadsheets/d/1Enaz3MGTjQgcSFGyBgc5YlFMz1n4HBvv727GW8JDxq8/edit?urlBuilderDomain=smartsetter.io#gid=1499399556",
        "https://docs.google.com/spreadsheets/d/11XWc7INBjz4cjjx_HGrn1Si1wIkODMhCZ7QkpQINxS0/edit#gid=1599873396",
        "https://docs.google.com/spreadsheets/d/18qNlTM-XhgId_hkkXFynWLpvHOzlVPXJM9cnbI52_Cw/edit?urlBuilderDomain=smartsetter.io#gid=223003133",
        "https://docs.google.com/spreadsheets/d/1I4RjcUzyCSmVmrJki40CmLBqMg-JmhS15saNcfi0ykU/edit?urlBuilderDomain=smartsetter.io#gid=923403582",
        "https://docs.google.com/spreadsheets/d/1o0AFjVAqC41BJZfeme90i-xve7jB_xA-864-ah4L3pU/edit?urlBuilderDomain=smartsetter.io#gid=752426163",
        "https://docs.google.com/spreadsheets/d/12JA_Bjrt-NHxb_01GPAOKPlG0wbOcMgeGE6bjh7_Ixw/edit?urlBuilderDomain=smartsetter.io#gid=1989394655",
        "https://docs.google.com/spreadsheets/d/1up3wlvKkQVwb4rolgm-OWXA4Brcvby1SJOv2VFfFZo8/edit?urlBuilderDomain=smartsetter.io#gid=1921068393",
        "https://docs.google.com/spreadsheets/d/1GoGL6F-HjGxhJX3tm8Mssigw2RmqenJ0vcpNY7DBG0A/edit?urlBuilderDomain=smartsetter.io#gid=894022260",
        "https://docs.google.com/spreadsheets/d/1iwETBuI5YgOwWjn5r564jcvTIElfULZ4lHHkZPceGJ0/edit?urlBuilderDomain=smartsetter.io#gid=1412264867",
        "https://docs.google.com/spreadsheets/d/1TvtahaiS4JWkz-MHop4FmGviepBO1Eu7dTHDLIbON3E/edit?urlBuilderDomain=smartsetter.io#gid=347822419",
        "https://docs.google.com/spreadsheets/d/1FWpo01xJpIfWwx4yIW2S1evl4PSDcY1n6loBmuq3M9g/edit?urlBuilderDomain=smartsetter.io#gid=741166654"]


In [ ]:
#urls = ["https://docs.google.com/spreadsheets/d/1mwa8slB8eZQTF6cS5bCEoCfGF8QeSA8w_lexb3Wi0vE/edit#gid=260483104"]

In [ ]:
#To get spreadsheet informations like spreadsheet file name and sheet name
spreadsheet_id = []
sheet_id = []
file_name = []
sheet_name = []
for sid in urls:
    spread_id = sid.split('/')
    spreadsheet_id.append(spread_id[5])
    sh_id = sid.split('=')
    sheet_id.append(sh_id[-1])
    sheet_metadata = service.spreadsheets().get(spreadsheetId=spread_id[5]).execute()
    file_name.append(sheet_metadata.get('properties').get('title'))
    properties = sheet_metadata.get('sheets')
    
    for  item in properties:
        if item.get("properties").get('sheetId') == int(sh_id[-1]):
            sh_name = (item.get("properties").get('title'))
            sheet_name.append(sh_name)

In [ ]:
#For reading all the files and creating dynamic variable for each dataframe
column_length = []
for url in range(len(urls)):
    
    globals()[f"df{url}"] = res.read_dataframe(urls[url])
    
    print(globals()[f"df{url}"].shape)
    column_length.append(globals()[f"df{url}"].shape[1])
    globals()[f"df{url}"]["Full_Name"] = None
    globals()[f"df{url}"]["Found_Mobile"] = None
    globals()[f"df{url}"]["Assign"]  = None 
    #globals()[f"df{url}"]["Comments"]  = None 

(1524, 15)
(1541, 15)
(1238, 15)
(1008, 15)
(3729, 15)
(4893, 15)
(2561, 15)
(1215, 15)
(6503, 15)
(2250, 15)
(1415, 15)
(6952, 15)
(3300, 15)
(2336, 15)
(7506, 15)
(2363, 15)
(4023, 15)
(2807, 15)


In [ ]:
#finding all unique states and creating state file #extracting state file from DB
cursor = connection.cursor()
state = []

for url in range(len(urls)):
    
    print("\nTaking state name for file name --->  ", file_name[url], "sheet name -->", sheet_name[url])
    find_state = res.state_finder(globals()[f"df{url}"].columns)
    if len(find_state) != 0:
        for uniq_state in  globals()[f"df{url}"][find_state].unique():
            if uniq_state not in state:
                if str(uniq_state).lower()!= 'nan':
                    state.append(uniq_state)
    else:
        print("couldn't find state column in this file no : ", url)
for st in state:
    
    postgreSQL_select_Query = 'select * from "manual_staging" where "state" = '+"'"+str(st)+"'"+' '
    cursor.execute(postgreSQL_select_Query)
    records = cursor.fetchall()
    column_list = [i.name for i in cursor.description]
    globals()[f"{st}"] = pd.DataFrame(records,columns=column_list)
#     globals()[f"{st}"] = history_file[history_file["state"]==st]
#     globals()[f"{st}"].reset_index(drop=True,inplace = True)
        
connection.close()
cursor.close()  
print("\n All the states taken and extracted state file from DB")
print("\n states ---> ",state)



Taking state name for file name --->   Master Leads | Tom Jervis sheet name --> Enrichment
Taking office_state as state 

Taking state name for file name --->   Master Leads | Susan Marcrie sheet name --> ENRICHMENT
Taking office_state as state 

Taking state name for file name --->   Master Leads | Thom Dallman sheet name --> Enrichment
Taking office_state as state 

Taking state name for file name --->   Master Leads | Kevin Sigstad sheet name --> ENRICHMENT 2
Taking office_state as state 

Taking state name for file name --->   Master Leads | Lindsay Sanger sheet name --> Enrichment
Taking office_state as state 

Taking state name for file name --->   Master Leads | Curtis Johnson and Ahmad Essawy sheet name --> Enrichment (Elastic)
Taking office_state as state 

Taking state name for file name --->   Master Leads | Beth Scofield sheet name --> Enrichment
Taking office_state as state 

Taking state name for file name --->   Master Leads | Brian Kwilosz sheet name --> Enrichment
Tak

In [ ]:
df0.columns

Index(['id', 'full_name', 'zip', 'cell', 'email', 'brand', 'position',
       'office_name', 'office_address', 'office_city', 'office_state',
       'office_zip', 'first_name', 'last_name', 'license_id'],
      dtype='object')

In [ ]:
#For Matching Mobile number using Names and emails


reject_no = int(input("Enter how many files you want to reject "))

reject_files_no = []
if reject_no != 0:
    for reject in range(reject_no):
        reject_file = int(input("Enter the number of the file : "))
        if reject_file != 0:
            if reject_file-1 not in reject_files_no:
                reject_files_no.append(reject_file-1)

for url in range(len(urls)):
    
    if url in reject_files_no:
      continue
      
    Db_found_index = []
    
    globals()[f"df{url}"]["Full_Name"] = None
    globals()[f"df{url}"]["Found_Mobile"] = None
    globals()[f"df{url}"]["Assign"]  = None 
    #globals()[f"df{url}"]["Comments"]  = None 
    
    print("\n ************************************************************************** \n")
    print("Matching for file name : ",file_name[url], "sheet name -->", sheet_name[url])
    
    first_name , last_name  = res.Full_name_creator(globals()[f"df{url}"].columns)
    Agent_Name = ''
    
    if len(first_name) > 0  and len(last_name) > 0:
        pass
    else:
        Agent_Name = res.agent_name_finder(globals()[f"df{url}"].columns)
    
    if len(Agent_Name) > 0:
        globals()[f"df{url}"] = res.Name_checker(globals()[f"df{url}"],Agent_Name)
    
    full_name = res.full_name_finder(globals()[f"df{url}"].columns)
    state =  res.state_finder(globals()[f"df{url}"].columns)
    first_name, last_name = res.Full_name_creator(globals()[f"df{url}"].columns)
    email = res.emaiL_finder(globals()[f"df{url}"].columns)
    license = res.license_finder(globals()[f"df{url}"].columns)
    Zip = res.zip_finder(globals()[f"df{url}"].columns)
    
    if len(state) > 0:
      if len(email) > 0: #
          #globals()[f"df{url}"]["Found_Mobile"] = globals()[f"df{url}"].apply(lambda x: finding_mobile_email(x[email],x[state],x["Found_Mobile"],x[Zip]), axis=1)
          globals()[f"df{url}"]["Found_Mobile"] = globals()[f"df{url}"].apply(lambda x: finding_mobile_email(x[email],x[state],x["Found_Mobile"]), axis=1)


      if len(first_name) > 0 and len(last_name) > 0: #Extracting  concatenated Name [first name & last name]
          
          globals()[f"df{url}"]["Full_Name"] = globals()[f"df{url}"][first_name] +" "+ globals()[f"df{url}"][last_name]
          #globals()[f"df{url}"]["Found_Mobile"] = globals()[f"df{url}"].apply(lambda x: finding_mobile_name(x["Full_Name"],x[state],x["Found_Mobile"],x[Zip]), axis=1)
          globals()[f"df{url}"]["Found_Mobile"] = globals()[f"df{url}"].apply(lambda x: finding_mobile_name(x["Full_Name"],x[state],x["Found_Mobile"]), axis=1)


              
      if len(full_name) > 0: #Matching 
          
          #globals()[f"df{url}"]["Found_Mobile"] = globals()[f"df{url}"].apply(lambda x: finding_mobile_name(x[full_name],x[state],x["Found_Mobile"],x[Zip]), axis=1)
          globals()[f"df{url}"]["Found_Mobile"] = globals()[f"df{url}"].apply(lambda x: finding_mobile_name(x[full_name],x[state],x["Found_Mobile"]), axis=1)


              
      
      Db_found_index = globals()[f"df{url}"][globals()[f"df{url}"]["Found_Mobile"].notna()].index
      
      if len(Db_found_index) > 0:
          globals()[f"df{url}"].loc[Db_found_index,"Assign"] = "Matched From DB"

      
      globals()[f"df{url}"] = res.license_count(globals()[f"df{url}"],"Found_Mobile")
      
      
      if len(license)>0:
          
          globals()[f"df{url}"] = res.license_count(globals()[f"df{url}"],license)
          
          globals()[f"df{url}"] = res.license_count(globals()[f"df{url}"],"Found_Mobile")
          
          globals()[f"df{url}"] = res.duplicate_check(globals()[f"df{url}"])
          #globals()[f"df{url}"] = res.comment(globals()[f"df{url}"])

          globals()[f"df{url}"].drop(['license_id_count','Found_Mobile_count'],axis= 1 , inplace= True)
          
          

    
    

Enter how many files you want to reject 0

 ************************************************************************** 

Matching for file name :  Master Leads | Tom Jervis sheet name --> Enrichment
Taking full_name as full name 
Taking office_state as state 
Taking first_name as First name
Taking last_name as Last name
Taking email as Email
Taking  license_id as license
Taking  zip as zip
Taking  office_zip as zip

 ************************************************************************** 

Matching for file name :  Master Leads | Susan Marcrie sheet name --> ENRICHMENT
Taking full_name as full name 
Taking office_state as state 
Taking first_name as First name
Taking last_name as Last name
Taking email as Email
Taking  license_id as license
Taking  zip as zip
Taking  office_zip as zip

 ************************************************************************** 

Matching for file name :  Master Leads | Thom Dallman sheet name --> Enrichment
Taking full_name as full name 
Taking offi

### To Reject the unwanted files 

In [ ]:
#### To create report of what matched from the DB 
for url in range(len(urls)):

    try:
      existing_index = globals()[f"df{url}"][globals()[f"df{url}"]["cell"].notna()].index
      if len(existing_index) > 0:
          globals()[f"df{url}"].loc[existing_index,"Assign"] = "Existing value present"
    except:
      existing_index = []
      pass
    
    blank_index = globals()[f"df{url}"][globals()[f"df{url}"]["Assign"].isna()].index
    print("\n************* "+file_name[url]+" ************ File NO :",url+1,"\n")
    
    Total_count = len(globals()[f"df{url}"][globals()[f"df{url}"]["Found_Mobile"].notna()])
    Found_count = len(globals()[f"df{url}"][(globals()[f"df{url}"]["Found_Mobile"] !="Not Found") & (globals()[f"df{url}"]["Found_Mobile"].notna())])
    Not_Found = len(globals()[f"df{url}"][globals()[f"df{url}"]["Found_Mobile"]=="Not Found"])
    print("Existing value count : ", len(existing_index))
    print("Found Count : ", Found_count)
    print("Not Found Count : ",Not_Found)
    print("Total count[Found + Not Found] :", Total_count)
    
    print(len(blank_index),"Blanks left")
    
reject_no = int(input("Enter how many files you want to reject "))

reject_files_no = []
if reject_no != 0:
    for reject in range(reject_no):
        reject_file = int(input("Enter the number of the file : "))
        if reject_file != 0:
            if reject_file-1 not in reject_files_no:
                reject_files_no.append(reject_file-1)


************* Master Leads | Tom Jervis ************ File NO : 1 

Existing value count :  0
Found Count :  46
Not Found Count :  698
Total count[Found + Not Found] : 744
780 Blanks left

************* Master Leads | Susan Marcrie ************ File NO : 2 

Existing value count :  0
Found Count :  3
Not Found Count :  1457
Total count[Found + Not Found] : 1460
81 Blanks left

************* Master Leads | Thom Dallman ************ File NO : 3 

Existing value count :  0
Found Count :  0
Not Found Count :  1191
Total count[Found + Not Found] : 1191
47 Blanks left

************* Master Leads | Kevin Sigstad ************ File NO : 4 

Existing value count :  0
Found Count :  1
Not Found Count :  1007
Total count[Found + Not Found] : 1008
0 Blanks left

************* Master Leads | Lindsay Sanger ************ File NO : 5 

Existing value count :  0
Found Count :  679
Not Found Count :  26
Total count[Found + Not Found] : 705
3024 Blanks left

************* Master Leads | Curtis Johnson and

In [ ]:
#Automate file Allocation with Manual Count
for url in range(len(urls)):
    

    try:
        existing_index = globals()[f"df{url}"][globals()[f"df{url}"]["cell"].notna()].index
        if len(existing_index) > 0:
            globals()[f"df{url}"].loc[existing_index,"Assign"] = "Existing Number present"
    except:
        pass


    if url in reject_files_no:
        continue

    print("Assigning in the file no : ",url," file name --> ", file_name[url], " sheet name : " , sheet_name[url])
    print("Total Blanks left in this file ", len(blank_index))
    blank_index = globals()[f"df{url}"][globals()[f"df{url}"]["Assign"].isna()].index

    globals()[f"df{url}"] = res.Automate_File_Allocation(globals()[f"df{url}"])

In [ ]:
# #FOR AUTO ASSIGN

# person = []    #edit  
# person_count = []
# count = 0
# remaining_person_halfday = []
# remaining_person_count_halfday = []
# get_input = 1


# for url in range(len(urls)):

#     try:
#       existing_index = globals()[f"df{url}"][globals()[f"df{url}"]["cell"].notna()].index
#       if len(existing_index) > 0:
#           globals()[f"df{url}"].loc[existing_index,"Assign"] = "Existing value present"
#     except:
#       pass
    
#     if url in reject_files_no:
#         continue
    
    
#     full_day  = 0
#     half_day = 0
#     get_input = 1
#     print("\n****************************************************************************\n")
    
   
#     blank_index = globals()[f"df{url}"][globals()[f"df{url}"]["Assign"].isna()].index
    
    
#     print("Assigning in the file no : ",url," file name --> ", file_name[url], " sheet name : " , sheet_name[url])
#     print("Total Blanks left in this file ", len(blank_index))
#     file_count = len(blank_index)
    
#     if file_count ==  0:
#         continue
#     #****************************************input Full day*********************************************

#     analyst_name_fullday = []
    
#     if len(person) != 0 :
        
#         analyst_name_fullday.extend(person)
#         full_day = (len(person)-1)*500 + (500-person_count[0])

#         if globals()[f"df{url}"].shape[0] >= (len(person)-1)*500 + (500-person_count[0]):
            
#             get_input = 1
            
            
#         else:
            
#             get_input = 0
            
#     if len(remaining_person_halfday) != 0 :
#         half_day = (len(remaining_person_halfday)-1)*250 + (250 - remaining_person_count_halfday[0])
        
        
#     if file_count > full_day + half_day:
#         get_input = 1
#         if full_day + half_day != 0:
#             print(full_day + half_day , "Records will be assigned",person[:],remaining_person_halfday[:])
#             print(file_count - (full_day + half_day), "Records only left")
#     else:
#         get_input = 0
        
        
    
        
#     try:
        
#         if get_input == 1:
            
#             no_analyst_fullday = int(input("Enter the number of person are going to work for full day : "))
            
#         else:
            
#             no_analyst_fullday = 0
            
#     except:
        
#         print("\nEnter the valid number")
#         if get_input == 1:
#             no_analyst_fullday = int(input("Enter the number of person are going to work for full day : "))
#         else:
#             no_analyst_fullday = 0
            
#     if no_analyst_fullday > 0:
        
#         print("Enter the name of the people who are going to work for Full day")
#         for person_no in range(no_analyst_fullday):
#             name = str(input(str(person_no+1)+" Enter the Name :"))
#             analyst_name_fullday.append(name)
#         print("\n")
            
  
    
#     #****************************************Input half day*******************************************************
    
#     analyst_name_halfday = []
    
#     if len(remaining_person_halfday) != 0 :
#         analyst_name_halfday.extend(remaining_person_halfday)
        
        
#     try:
#         if get_input == 1:

#             no_analyst_halfday = int(input("Enter the number of person are going to work for half day  : "))
#         else:
#             no_analyst_halfday = 0
        
#     except:
#         print("Enter the valid input ")
        
#         if get_input == 1:
#             no_analyst_halfday = int(input("Enter the number of person are going to work for half day : "))
            
#         else:
#             no_analyst_halfday = 0

        
#     if no_analyst_halfday > 0:
#         print("Enter the name of the people who are going to work for half day ")
#         for person_no in range(no_analyst_halfday):
#             name = str(input(str(person_no+1)+" Enter the Name :"))
#             analyst_name_halfday.append(name)
#         print("\n")
    
    
    
#     #*****************************************************Full Day Assign*******************************************        


#     if len(analyst_name_fullday) != 0:
        
        
#         for name_no in range(len(analyst_name_fullday)):
            
#             if len(person) != 0 and name_no == 0 :
                
#                 count = person_count[0]
                
#             else:
                
#                 count = 0
                
#             globals()[f"df{url}"],count,status = res.assign(globals()[f"df{url}"],count, analyst_name_fullday[name_no])
            
#             print(count, "Assigned for ", analyst_name_fullday[name_no] )
            
            
#             if status == 'done':
                
                
#                 print("There is no blank left")
                
#                 #print(count, "Assigned for ", analyst_name_fullday[name_no] )
                
#                 print(500-count, "will be assigned in next file for ", analyst_name_fullday[name_no])

#                 if count < 500:
#                     if analyst_name_fullday[name_no] not in person:
#                         person.append(analyst_name_fullday[name_no])
#                         person_count.append(count)
                    
#                     if name_no == 0:
#                         person_count[0] = count
                        
                    
#                 if url == len(urls) - 1:

#                     if len(person) != 0:
                        
#                         print(person[:], "These persons are still remaining")
            
#             if status != "done":
#                 person = []
#                 person_count = []
                
                

#         #*******************************************************Half Day Assign*****************************************        


#     if len(analyst_name_halfday) != 0:
        
        
#         for name_no in range(len(analyst_name_halfday)):
            
#             if len(remaining_person_halfday) != 0 and name_no == 0 :
                
#                 count = remaining_person_count_halfday[0]
                
#             else:
                
#                 count = 0
                
#             globals()[f"df{url}"],count,status = res.assign_half(globals()[f"df{url}"],count, analyst_name_halfday[name_no])
            
#             print(count, "Assigned for ", analyst_name_halfday[name_no] )
            
            
#             if status == 'done':
                
                
#                 print("There is no blank left")
                
#                 #print(count, "Assigned for ", analyst_name_halfday[name_no] )
                
#                 print(250-count, "will be assigned in next file for", analyst_name_halfday[name_no])

#                 if count < 250:
#                     if analyst_name_halfday[name_no] not in person:
#                         remaining_person_halfday.append(analyst_name_halfday[name_no])
#                         remaining_person_count_halfday.append(count)
#                     if name_no == 0:
#                         remaining_person_count_halfday[0] = count
                        
                    
#                 if url == len(urls) - 1:

#                     if len(remaining_person_halfday) != 0:
                        
#                         print(remaining_person_halfday[:], "These persons are still remaining")
            
#             if status != "done":
#                 remaining_person_halfday = []
#                 remaining_person_count_halfday = []
                
#     try:
#         if globals()[f"df{url}"]["Assign"].isna().sum() > 0:
#             print(globals()[f"df{url}"]["Assign"].isna().sum(), "still left in this file")
#     except:
#         pass



****************************************************************************

Assigning in the file no :  0  file name -->  Master Leads | Jim Burton  sheet name :  ENRICHMENT TICKET (Elastic)
Total Blanks left in this file  2588
Enter the number of person are going to work for full day : 1
Enter the name of the people who are going to work for Full day
1 Enter the Name :Melvin


Enter the number of person are going to work for half day  : 0
500 Assigned for  Melvin
2088 still left in this file


### To Export to Google sheets 

In [ ]:
#To export to google sheets

albha = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","AA","AB","AC"]
for url in range(len(urls)):
    
    Data = globals()[f"df{url}"][["Full_Name","Found_Mobile","Assign"]]
    # na_index = Data[Data["Comments"].isna()].index
    # if len(na_index) > 0:
    #     Data.loc[na_index,"Comments"] = None
        
    na_index = Data[Data["Full_Name"].isna()].index
    if len(na_index) > 0:
        Data.loc[na_index,"Full_Name"] = None
       
    na_index = Data[Data["Found_Mobile"].isna()].index
    if len(na_index) > 0:
        Data.loc[na_index,"Found_Mobile"] = None
    
    na_index = Data[Data["Assign"].isna()].index
    if len(na_index) > 0:
        Data.loc[na_index,"Assign"] = None
    
    
    sheet_data_list = []
    for index in range(len(Data)):
        sheet_data_list.append([str(Data.loc[index,'Full_Name']),str(Data.loc[index,'Found_Mobile']), str(Data.loc[index,'Assign'])])
    


    data = [['Full_Name', 'Found_Mobile', 'Assign']]
    request = service.spreadsheets().values().append(spreadsheetId=spreadsheet_id[url], 
                                                     range=sheet_name[url]+"!"+albha[column_length[url]]+str(1)+":"+albha[column_length[url]+2]+str(1), valueInputOption="USER_ENTERED", 
                                                     insertDataOption="OVERWRITE", body={"values":data})

    response = request.execute()
    
        
   



    data = sheet_data_list
    request = service.spreadsheets().values().append(spreadsheetId=spreadsheet_id[url], 
                                                     range=sheet_name[url]+"!"+albha[column_length[url]]+str(2)+":"+albha[column_length[url]+2]+str(2), valueInputOption="USER_ENTERED", 
                                                     insertDataOption="OVERWRITE", body={"values":data})

    response = request.execute()

NameError: ignored